In [6]:
import csv
import plotly.graph_objects as go
import plotly.subplots as sp
import pandas as pd

In [7]:

# Import the file
df = pd.read_csv('./data/simulated_shading.csv')
# Get a list of module_2 column
# module_2 = df['module_2'].tolist()
# module_2
df['bypass_on'].unique()

array([0.401, 0.796, 1.202, 0.794, 1.186, 1.2  ])

In [8]:
# Plots a 3d plot with module_1 as x-axis, module_2 as y-axis and the bypass_on column on the z-axis in red and bypass_off column on the z-axis in red
fig = go.Figure(data=[go.Scatter3d(
    x=df['module_1'],
    y=df['module_2'],
    z=df['bypass_on'],
    mode='markers',
    marker=dict(
        size=5,
        color='red',                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
), go.Scatter3d(
    x=df['module_1'],
    y=df['module_2'],
    z=df['bypass_off'],
    mode='markers',
    marker=dict(
        size=5,
        color='blue',                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)])
# Update size to 1200x1200
# Label the axes
fig.update_layout(scene=dict(
                    xaxis_title='Module 1',
                    yaxis_title='Module 2',
                    zaxis_title='Bypass'),
                    width=1200,
                    height=1200)
# Only show the tickvals on the x and y axes with the data
# Show the figure
fig.show()



In [9]:
# Plots a 3d plot with module_1 as x-axis, module_2 as y-axis and the bypass_on column on the z-axis in red and bypass_off column on the z-axis in red
fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=df['module_1'],
    y=df['module_2'],
    z=df['bypass_off'],
    mode='markers',
    name = 'Bypass Off',
    marker=dict(
        size=8,
        color='blue',                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
))
fig.add_trace(go.Scatter3d(
    x=df['module_1'],
    y=df['module_2'],
    z=df['bypass_on'],
    mode='markers',
    name='Bypass On',
    marker=dict(
        size=8,
        color='red',                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
))

# Extract unique values for tickvals
x_tickvals = df['module_1'].unique()
y_tickvals = df['module_2'].unique()
z_tickvals = pd.concat([df['bypass_on'], df['bypass_off']]).unique()
# Sort the z_tickvals array
z_tickvals.sort()

clusters = []
current_cluster = [z_tickvals[0]]

# Group values into clusters
for i in range(1, len(z_tickvals)):
    if z_tickvals[i] - current_cluster[-1] <= 0.05:
        current_cluster.append(z_tickvals[i])
    else:
        clusters.append(current_cluster)
        current_cluster = [z_tickvals[i]]

# Add the last cluster if it wasn't added in the loop
if current_cluster:
    clusters.append(current_cluster)

# Pick the highest value from each cluster
pruned_z_tickvals = [max(cluster) for cluster in clusters]

z_tickvals = pruned_z_tickvals

# Label the axes
fig.update_layout(scene=dict(
                    xaxis_title='Module 1 Limit [A]',
                    yaxis_title='Module 2 Limit [A]',
                    zaxis_title='Array Current [A]',
                    xaxis=dict(tickvals=x_tickvals),
                    yaxis=dict(tickvals=y_tickvals),
                    zaxis=dict(tickvals=z_tickvals))
                    )


# Show the figure
fig.show()


In [10]:
import numpy as np
from sklearn.linear_model import LinearRegression


# Fit a regression plane for bypass_on (red points)
X_on = df[['module_1', 'module_2']]
y_on = df['bypass_on']
reg_on = LinearRegression().fit(X_on, y_on)
xx_on, yy_on = np.meshgrid(df['module_1'].unique(), df['module_2'].unique())
zz_on = reg_on.coef_[0] * xx_on + reg_on.coef_[1] * yy_on + reg_on.intercept_

# Add the regression plane for bypass_on to the figure
fig.add_trace(go.Surface(x=xx_on, y=yy_on, z=zz_on, colorscale='Reds', opacity=0.5, name='Regression Plane (bypass_on)'))

# Fit a regression plane for bypass_off (blue points)
X_off = df[['module_1', 'module_2']]
y_off = df['bypass_off']
reg_off = LinearRegression().fit(X_off, y_off)
xx_off, yy_off = np.meshgrid(df['module_1'].unique(), df['module_2'].unique())
zz_off = reg_off.coef_[0] * xx_off + reg_off.coef_[1] * yy_off + reg_off.intercept_

# Add the regression plane for bypass_off to the figure
fig.add_trace(go.Surface(x=xx_off, y=yy_off, z=zz_off, colorscale='Blues', opacity=0.5, name='Regression Plane (bypass_off)'))

# Move the legend to the top left corner
fig.update_layout(legend=dict(x=0, y=1, traceorder='normal', font=dict(size=16)))
# Update title to Simulated Shading with 2 modules
fig.update_layout(title='Simulated Shading with 2 Modules')
# Update title font to 24
fig.update_layout(title_font_size=24)
# Update font size to 26
fig.update_layout(scene=dict(
                    xaxis=dict(title_font_size=22),
                    yaxis=dict(title_font_size=22),
                    zaxis=dict(title_font_size=22)))
# Update tickval font to 18
fig.update_layout(scene=dict(
                    xaxis=dict(tickfont_size=16),
                    yaxis=dict(tickfont_size=16),
                    zaxis=dict(tickfont_size=15)))
# Change size to 1200x1200
fig.update_layout(width=800, height=800)
# Show the figure
fig.show()
fig.write_html("./data/simualtedshade.html")
